In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
import keras
keras.__version__

'2.7.0'

In [4]:
import tensorflow as tf
tf.__version__

'2.7.2'

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten,BatchNormalization,Dropout

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IgnIGX_aPuKu4DHFTUABCggxOUHZG6JoBBOlueiypNXO',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ecgclassification-donotdelete-pr-fhxyyoyjwxtmly'
object_key = 'Dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
if not hasattr(streaming_body_3, "__iter__"): streaming_body_3.__iter__ = types.MethodType( __iter__, streaming_body_3 )

In [9]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [10]:
import os
filename =os.listdir('/home/wsuser/work/Dataset/Test')

In [11]:
x_train = train_datagen.flow_from_directory(directory=r'/home/wsuser/work/Dataset/Train',target_size = (64,64),batch_size = 32,class_mode = "categorical")
x_test = test_datagen.flow_from_directory(directory=r'/home/wsuser/work/Dataset/Test',target_size = (64,64),batch_size = 32,class_mode = "categorical")

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [12]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

In [13]:
model = Sequential()


In [14]:
model.add(Convolution2D(32,(3,3),padding="same",input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Convolution2D(32,(3,3),padding="same",input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
model.add(Convolution2D(32,(3,3),padding="same",input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
model.add(Flatten()) # ANN Input...

In [18]:
model.add(Dense(units=128,kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),activation="elu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

In [19]:
model.add(Dense(units=128,kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),activation="elu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

In [20]:
model.add(Dense(units=64,kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),activation="elu"))
model.add(Dropout(0.2))

In [21]:
model.add(Dense(units=64,kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),activation="elu"))
model.add(Dropout(0.2))

In [22]:
model.add(Dense(units = 6,kernel_initializer = "random_uniform",activation = "softmax"))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 32)         0

In [24]:
from tensorflow.keras.optimizers import RMSprop , SGD , Adam
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau

In [25]:
reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=2,
                             verbose=1,
                             min_delta=0.0001)

In [27]:
callbacks=[reduce_lr]
model.compile(loss="categorical_crossentropy",optimizer=Adam(lr=0.01,beta_1=0.1),metrics=['accuracy'])
batch_size=3

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [28]:
model.fit_generator(x_train,steps_per_epoch=len(x_train)/batch_size,epochs=10,callbacks=callbacks,validation_data=x_test,validation_steps=10)

/tmp/wsuser/ipykernel_5366/3452490920.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train)/batch_size,epochs=10,callbacks=callbacks,validation_data=x_test,validation_steps=10)


Epoch 1/10
160/160 [==============================] - 33s 202ms/step - loss: 0.9570 - accuracy: 0.6844 - val_loss: 176.9642 - val_accuracy: 0.1281 - lr: 0.0100
Epoch 2/10
160/160 [==============================] - 32s 203ms/step - loss: 0.5792 - accuracy: 0.8161 - val_loss: 121.1843 - val_accuracy: 0.1844 - lr: 0.0100
Epoch 3/10
160/160 [==============================] - 32s 201ms/step - loss: 0.4033 - accuracy: 0.8765 - val_loss: 61.4337 - val_accuracy: 0.1562 - lr: 0.0100
Epoch 4/10
160/160 [==============================] - 32s 203ms/step - loss: 0.4858 - accuracy: 0.8510 - val_loss: 540.6391 - val_accuracy: 0.3250 - lr: 0.0100
Epoch 5/10
160/160 [==============================] - ETA: 0s - loss: 0.3897 - accuracy: 0.8785
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
160/160 [==============================] - 32s 199ms/step - loss: 0.3897 - accuracy: 0.8785 - val_loss: 144.7920 - val_accuracy: 0.3187 - lr: 0.0100
Epoch 6/10
160/160 [================

In [29]:
model.save('ECG.h5')

In [30]:
!tar -zcvf ECG-model_new.tgz ECG.h5

ECG.h5


In [31]:
ls -1

Dataset/
ECG.h5
ECG-model_new.tgz
__MACOSX/


In [32]:
pip install watson-machine-learning-client --upgrade

Note: you may need to restart the kernel to use updated packages.


In [33]:
# Replace the credentials that you got from watson machine learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"rkRoqLY3k9Cyqe9Toeom5WfwuHmC6yeVfSW0zks-N9UD"
                  }
client = APIClient(wml_credentials)

In [34]:
client = APIClient(wml_credentials)

In [35]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [36]:
space_uid = guid_from_space_name(client, 'ecg_classification')
print("Space UID = "+ space_uid)

Space UID = 111b1766-3695-488a-83a2-99add28cae4e


In [37]:
client.set.default_space(space_uid)

'SUCCESS'

In [38]:
client.software_specifications.list(limit=100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [39]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [45]:
model_details = client.repository.store_model(model='ECG-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"ECG_Model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_id(model_details)

In [46]:
model_id

'9e82f0f5-b46b-4780-b0cb-f2fb09b1bb61'

In [47]:
client.repository.download(model_id,'my_modell.tar1.gz')

Successfully saved model content to file: 'my_modell.tar1.gz'


'/home/wsuser/work/my_modell.tar1.gz'

In [48]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [49]:
model = load_model("ECG.h5")

In [52]:
pwd


'/home/wsuser/work'